In [2]:
import os
import json
import pickle
import numpy as np
import seaborn as sns
from tqdm import tqdm
import pandas as pd
# from dance_evaluation import *
import matplotlib.pyplot as plt
# from calculate_score import *
from collections import defaultdict
from utils.eval import *

In [2]:
def compute_dts(
    ref_bpm,
    estimated_bpm,
    tau=0.13,
    mode="one"
):
    """
    Continuous Dance-Tempo Score (DTS), with support for
    either single estimates (mode="one") or multiple
    candidates per frame (mode="many").

    Parameters
    ----------
    ref_bpm : array-like, shape (n,)
        Ground-truth musical tempo in BPM.
    estimated_bpm : 
        If mode="one": array-like, shape (n,)
        If mode="many": iterable of length-n, each element
                        is an iterable of candidate BPMs.
    tau : float, optional
        Tolerance in octaves (0.06 ≈ 4 %).
    mode : {"one", "many"} 
        “one”: treat `estimated_bpm` as a flat sequence.
        “many”: pick, for each i, the candidate closest to ref_bpm[i]. For best of two

    Returns
    -------
    dts : ndarray, shape (n,)
        Scores in [0, 1] (1 = perfect, 0 = miss ≥ τ octaves away).
    e : ndarray, shape (n,)
        Raw octave errors log₂(estimate/ref).
    d : ndarray, shape (n,)
        Wrapped distance to {-1, 0, +1} before clipping.
    """
    ref_bpm = np.asarray(ref_bpm, dtype=float)

    body_parts = ["hand", "foot", "torso"]

    if mode == "many":
        chosen = []
        for i, cands in enumerate(estimated_bpm):  # e.g. (bpm_hand, bpm_foot, bpm_torso)
            ref = ref_bpm[i]
            diffs = [
                min(abs(b - ref), abs(b - 0.5 * ref), abs(b - 2.0 * ref))
                for b in cands
            ]
            idx_min = int(np.argmin(diffs))  # index of best match
            chosen_bpm = cands[idx_min]
            chosen_part = body_parts[idx_min]
            chosen.append((chosen_bpm, chosen_part))

    elif mode == "one":
        chosen = [(float(b), None) for b in np.asarray(estimated_bpm, dtype=float)]
    else:
        raise ValueError(f"Unknown mode: {mode!r}. Use 'one' or 'many'.")

    chosen_bpm = np.array([c[0] for c in chosen], dtype=float)
    # DTS core ------------------------------------------------------
    e = np.log2(chosen_bpm / ref_bpm)
    # distance from nearest of -1, 0, +1
    d = np.abs(e[:, None] - np.array([-1.0, 0.0, 1.0])).min(axis=1)
    # clip by tolerance and convert to score
    d_clip = np.minimum(d, tau)
    dts    = 1.0 - d_clip / tau

    accuracy = (dts > 0.0).mean() * 100
    
    # hits ----------------------------------------------------------
    hit_mask = dts > 0.0          # inside ±tau band
    hit_idx = np.nonzero(hit_mask)[0]
    ref_hit_bpm = ref_bpm[hit_idx]
    
    return accuracy, hit_idx, ref_hit_bpm, chosen

### Functions

In [9]:
def estimate_tempo_posvel(a, b, mode, w_sec, h_sec, tolerance=0.13):
    # Using both zero velocity and peak velocity
    segment_pairs = [
        ("both_hand_x", "both_foot_x"),                     # Example: bpm_pos ~35%, bpm_posvel ~55%
        ("both_hand_y", "both_foot_y"),                     # Example: bpm_pos ~70%, bpm_posvel ~81%
        ("both_hand_resultant", "both_foot_resultant"),     # Example: bpm_pos ~49%, bpm_posvel ~66%

    ]
    
    score_data = {}
    json_data = {}
    oPath = f"./saved_result/tempo_{a}_{b}/"



    bpm_dict = ["bpm_median"]
    for bpm_mode in bpm_dict:
        score_data[bpm_mode] = {}
        json_data[bpm_mode] = {}
        for hnd, ft in segment_pairs:
            # Build file paths for position and velocity data
            read_file1 = oPath + f"pos/{hnd}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
            read_file2 = oPath + f"pos/{ft}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
            read_file3 = oPath + f"vel/{hnd}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
            read_file4 = oPath + f"vel/{ft}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
            read_file5 = oPath + f"pos/torso_y_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"   # for reference tempo
            
            # Load the dataframes
            df1 = pd.read_pickle(read_file1)
            df2 = pd.read_pickle(read_file2)
            df3 = pd.read_pickle(read_file3)
            df4 = pd.read_pickle(read_file4)
            df5 = pd.read_pickle(read_file5)   # for reference tempo
            
            # Build candidate BPM pairs (for positions) and quads (for positions and velocities)
            bpm_pos = []
            bpm_vel = []
            bpm_posvel = []
            for n in range(df1.shape[0]):
                bpm1 = df1.iloc[n][bpm_mode]   # hand (position)
                bpm2 = df2.iloc[n][bpm_mode]   # foot (position)
                bpm3 = df3.iloc[n][bpm_mode]   # hand (velocity)
                bpm4 = df4.iloc[n][bpm_mode]   # foot (velocity)
                bpm5 = df5.iloc[n][bpm_mode]   # torso (position) - not used here
                
                bpm_pos.append((bpm1, bpm2))
                bpm_vel.append((bpm3, bpm4))
                bpm_posvel.append((bpm1, bpm2, bpm5))     # , bpm5
                # bpm_posvel.append((bpm1, bpm2, bpm3, bpm4))     # , bpm5
                # bpm_posvel.append((bpm1, bpm2, bpm3, bpm4, bpm5))     # , bpm5
            
            # music_tempo from df1 
            ref = df1["music_tempo"].to_numpy()
            
            # Calculate metrics for the candidate
            dts_acc1, hit_idx1, ref_hit_bpm1,_ = compute_dts(ref, bpm_pos, tau=tolerance, mode = "many")
            dts_acc2, hit_idx2, ref_hit_bpm2,_ = compute_dts(ref, bpm_vel, tau=tolerance, mode = "many")
            dts_acc3, hit_idx3, ref_hit_bpm3, chosen = compute_dts(ref, bpm_posvel, tau=tolerance, mode = "many")

            pair_key = f"{hnd}_{ft}"
            
            score_data[bpm_mode][pair_key] = {"metrics_pos": hit_idx1, "metrics_vel": hit_idx2, "metrics_posvel": hit_idx3}
            
            
            json_data[bpm_mode][pair_key] = {
                                            "ref": ref,
                                            "bpm_pos": bpm_pos,
                                             "bpm_vel": bpm_vel,
                                            "bpm_posvel": bpm_posvel,
                                            "Acc1_bpm_pos": dts_acc1,
                                            "Acc1_bpm_vel": dts_acc2,
                                            "Acc1_bpm_posvel": dts_acc3,
                                            }
            
    
    #### Sace the score data to a pickle file
    # save_dir = f"./saved_result/tempo_{a}_{b}/score"
    # fname1 = f"score_multi_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
    # fpath1 = os.path.join(save_dir, fname1)
    # save_to_pickle(fpath1, score_data)
        
    return json_data, chosen, ref, hit_idx3

def estimate_tempo_one(a, b, mode, metric, w_sec, h_sec, tolerance=0.13):

    segment_keys = [
                    "torso_y",
                    "left_hand_x", "right_hand_x", "left_hand_y", "right_hand_y",   # singular
                    "left_foot_x", "right_foot_x", "left_foot_y", "right_foot_y",   # singular
                    
                    "lefthand_xy", "righthand_xy", "leftfoot_xy", "rightfoot_xy",   # singular | 35, 40, 34, 36
                    "left_hand_resultant", "right_hand_resultant", "left_foot_resultant", "right_foot_resultant",   # singular | 18,20,17,17 %
                    
                    "both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y",
                     "both_hand_resultant", "both_foot_resultant", # resultant of x and y onsets
                    
                    "bothhand_x_bothfoot_x", "bothhand_y_bothfoot_y",
                    "lefthand_xy_righthand_xy", "leftfoot_xy_rightfoot_xy",
                    "bothhand_x_bothhand_y", "bothfoot_x_bothfoot_y", 
                    ] 
    
    score_data = {}
    json_data = {}
    bpm_dict = ["bpm_median"]
    oPath = f"./tempo_estimation_output/tempo_{a}_{b}/"
    for bpm_mode in bpm_dict:
        score_data[bpm_mode] = {}
        json_data[bpm_mode] = {}
        for idx, f_name in enumerate(segment_keys):
            f_path = oPath + f"{metric}/{f_name}_{mode}.pkl"        # _W{w_sec}_H{h_sec}_{a}_{b}
            df_ax = pd.read_pickle(f_path)
            
            ref = df_ax["music_tempo"].to_numpy()
            dts_acc, hit_idx, ref_hit_bpm = compute_dts(ref, np.asarray(df_ax[bpm_mode]), tau=tolerance, mode = "one")
            
            score_data[bpm_mode][f_name] = {f"metrics_{metric}": hit_idx}
                                    
            json_data[bpm_mode][f_name] = dts_acc   #{"Acc1_bpm_one": dts_acc}
            # json_data[bpm_mode][f_name] = metrics     # new
            
    
    #### Save the score data to a pickle file
    # save_dir = f"./saved_result/tempo_{a}_{b}/score"
    # fname1 = f"score_one_{metric}_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl"
    # fpath1 = os.path.join(save_dir, fname1)
    # save_to_pickle(fpath1, score_data)
    
    return json_data

a = 45; b= 140
mode = "uni"
w_sec = 5; h_sec = w_sec/2

# jdata1, chosen, ref, hit_idx3 = estimate_tempo_posvel(a, b, mode, w_sec, h_sec, tolerance=0.13)      # overall acc1 per body segment
jdata2 = estimate_tempo_one(a, b, mode, "anchor_zero", w_sec, h_sec, tolerance=0.13)    # overall acc1 per body segment
# jdata3 = estimate_tempo_one(a, b, mode, "vel", w_sec, h_sec, tolerance=0.13)    # overall acc1 per body segment

In [10]:
jdata2["bpm_median"]

{'torso_y': 61.96868008948546,
 'left_hand_x': 44.369873228933635,
 'right_hand_x': 44.1461595824012,
 'left_hand_y': 58.76211782252051,
 'right_hand_y': 57.867263236390755,
 'left_foot_x': 38.62788963460104,
 'right_foot_x': 42.132736763609245,
 'left_foot_y': 47.80014914243102,
 'right_foot_y': 48.02386278896346,
 'lefthand_xy': 47.35272184936615,
 'righthand_xy': 51.30499627143923,
 'leftfoot_xy': 40.26845637583892,
 'rightfoot_xy': 44.593586875466066,
 'left_hand_resultant': 35.57046979865772,
 'right_hand_resultant': 39.299030574198355,
 'left_foot_resultant': 35.272184936614465,
 'right_foot_resultant': 36.6144668158091,
 'both_hand_x': 34.22818791946309,
 'both_hand_y': 63.534675615212535,
 'both_foot_x': 30.499627143922446,
 'both_foot_y': 54.660700969425804,
 'both_hand_resultant': 37.28560775540642,
 'both_foot_resultant': 43.475018642803875,
 'bothhand_x_bothfoot_x': 33.03504847129008,
 'bothhand_y_bothfoot_y': 51.67785234899329,
 'lefthand_xy_righthand_xy': 46.3832960477255

In [ ]:
a = 60; b= 140
mode = "zero_uni"
w_sec = 5; h_sec = w_sec/2

t_data = estimate_tempo_posvel(a, b, mode, w_sec, h_sec, tolerance=0.13)
# jdata2 = estimate_tempo_one(a, b, mode, "pos", w_sec, h_sec)
# jdata2["bpm_median"]["torso_y"]["Acc1_bpm_one"]

## Find dominant body part per genre

In [ ]:
import pandas as pd
from collections import defaultdict


with open("genre_symbols_mapping.json", "r") as file:
    genre_name = json.load(file)


read_dir = f"./saved_result/tempo_{a}_{b}"
fpath4 = os.path.join(read_dir, f"pos/left_hand_y_{mode}_W{w_sec}_H{h_sec}_{a}_{b}.pkl")
main_df = pd.read_pickle(fpath4)


# Assuming chosen and main_df have the same length
genre_part_map = defaultdict(list)

for i, (bpm, part) in enumerate(chosen):
    genre = main_df.loc[i, "dance_genre"]
    genre_part_map[genre_name[genre]].append(part)


# Convert to a DataFrame for easier counting
genre_part_df = pd.DataFrame([
    {"genre": genre, "body_part": part}
    for genre, parts in genre_part_map.items()
    for part in parts
])

# Count frequency of each body part per genre
part_counts = (
    genre_part_df.groupby(["genre", "body_part"])
    .size()
    .reset_index(name="count")
    .sort_values(["genre", "count"], ascending=[True, False])
)
print(part_counts)


In [ ]:
import numpy as np

# Example data
# ref_bpm = np.array([120, 100, 140])
# est_bpm = [(60, 119, 240), (98, 200), (70, 280)]

half_count = 0
double_count = 0
half_matches = []

for ref, cands in zip(ref_bpm, est_bpm):
    cands = np.array(cands, dtype=float)
    if np.any(np.isclose(cands, 0.5 * ref, rtol=0.1)):
        half_count += 1
        half_matches.append(ref)
    if np.any(np.isclose(cands, 2.0 * ref, rtol=0.1)):
        double_count += 1

print(f"Half-time matches: {half_count}")
print(f"Double-time matches: {double_count}")



In [ ]:
max(half_matches)

### Bar Plot: Both Hand+Foot (Multi Score)

In [ ]:
segments_3 = [
        'both_hand_x_both_foot_x', 'both_hand_y_both_foot_y', 
        'both_hand_resultant_both_foot_resultant'
        ]   # , 'bothhand_xy_bothfoot_xy'

a = 60; b= 140
mode = "zero_uni"
w_sec = 5; h_sec = w_sec/2

t_data = estimate_tempo_posvel(a, b, mode, w_sec, h_sec, tolerance=0.13)
accuracies_pos = [round(t_data['bpm_median'][seg]['Acc1_bpm_pos']) for seg in segments_3]
accuracies_vel = [round(t_data['bpm_median'][seg]['Acc1_bpm_vel']) for seg in segments_3]
accuracies_posvel = [round(t_data['bpm_median'][seg]['Acc1_bpm_posvel']) for seg in segments_3]

labels = [
    'Hand_foot_X', 'Hand_foot_Y','Hand_foot_res', 
]   # , 'Hand_foot_XY' 

# Define colors for each metric as separate variables
pos_color    = '#AEC6CF'   # Color for Position-based accuracy
vel_color    = '#C3B1E1'   # Color for Velocity-based accuracy
posvel_color = '#FDFD96'   # Color for Combined PosVel accuracy

# Set up x-axis positions for groups
x = np.arange(len(labels))
bar_width = 0.2
inner_spacing = 0.05

# Calculate positions for each of the three bars per group
x_pos    = x - (bar_width + inner_spacing/2)
x_vel    = x
x_posvel = x + (bar_width + inner_spacing/2)

# Create the grouped bar plot
plt.figure(figsize=(8, 5))

bars_pos    = plt.bar(x_pos,    accuracies_pos,    width=bar_width, color=pos_color,    edgecolor='gray', label='Position')
bars_vel    = plt.bar(x_vel,    accuracies_vel,    width=bar_width, color=vel_color,    edgecolor='gray', label='Velocity')
bars_posvel = plt.bar(x_posvel, accuracies_posvel, width=bar_width, color=posvel_color, edgecolor='gray', label='PosVel')

plt.xticks(x, labels, rotation=60, ha='right')
plt.ylabel('Accuracy (%)')
plt.title('Best of ')
plt.ylim(0, 100)
plt.legend(title="Metric")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Annotate each bar with its value
for bar in bars_pos + bars_vel + bars_posvel:
    plt.text(
        bar.get_x() + bar.get_width()/2,
        bar.get_height() + 1,
        f'{bar.get_height()}%',
        ha='center',
        fontsize=9
    )

plt.tight_layout()
plt.show()

# Plot Section

### V2: Grouped Bar Plot: Tempo of independant body segment

In [ ]:
# Your parameters
a, b = 60, 140
metric = "pos"
w_sec, h_sec = 5, 5/2

segments = [
    "left_hand_x", "left_hand_y", "right_hand_x", "right_hand_y",
    "left_foot_x", "left_foot_y", "right_foot_x", "right_foot_y",
]

labels = [
    'L-Hand-X', 'L-Hand-Y', 'R-Hand-X', 'R-Hand-Y',
    'L-Foot-X', 'L-Foot-Y', 'R-Foot-X', 'R-Foot-Y',
]

# Compute accuracies for each mode
modes = ["zero_uni", "zero_bi"]    
acc = {}
for mode in modes:
    t_data = estimate_tempo_one(a, b, mode, metric, w_sec, h_sec, tolerance=0.13)
    acc[mode] = np.array([
        round(t_data["bpm_median"][seg]['Acc1_bpm_one'])
        for seg in segments
    ])

# Plotting
bar_width = 0.35
x = np.arange(len(labels))

plt.figure(figsize=(10, 4), dpi=100)
plt.bar(x - bar_width/2, acc["zero_uni"],  bar_width, label="Uni-directional", color = "tab:blue")
plt.bar(x + bar_width/2, acc["zero_bi"],  bar_width, label="Bi-directional", color = "tab:orange")

plt.xticks(x, labels, rotation=60)
plt.ylabel("Accuracy (%)")
plt.ylim(0, 100)
plt.title("Tempo Estimation Accuracy Across Single Body")
plt.legend()

# annotate
for i in range(len(x)):
    plt.text(x[i] - bar_width/2, acc["zero_uni"][i] + 1,
             f"{acc['zero_uni'][i]}%", ha="center", va="bottom")
    plt.text(x[i] + bar_width/2, acc["zero_bi"][i] + 1,
             f"{acc['zero_bi'][i]}%", ha="center", va="bottom")

plt.tight_layout()
# plt.savefig("plots/plot_1a_peak_vel.png")
plt.show()


### V2: Bar plot: XY and resultant

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Your fixed parameters
a, b       = 60, 140
mode       = "zero_uni"
w_sec, h_sec = 5, 5/2

segments = [
    "lefthand_xy", "righthand_xy", "leftfoot_xy", "rightfoot_xy",
    "left_hand_resultant", "right_hand_resultant",
    "left_foot_resultant", "right_foot_resultant",
]
labels = [
    'L-Hand-XY', 'R-Hand-XY', 'L-Foot-XY', 'R-Foot-XY',
    'L-Hand-Res','R-Hand-Res','L-Foot-Res','R-Foot-Res'
]

# Compute accuracies for both metrics
acc = {}
for metric in ("pos", "vel"):
    t_data    = estimate_tempo_one(a, b, mode, metric, w_sec, h_sec, tolerance=0.13)
    acc[metric] = np.array([
        round(t_data['bpm_median'][seg]['Acc1_bpm_one'])
        for seg in segments
    ])


# Plot setup
# gap   = 0.2                         # reduce this to bring bars closer
# group = bar_width*2 + gap          # two bars per group + small gap
# x = np.arange(len(labels)) * group
# x = np.arange(len(labels)) * (bar_width*2 + 0.2)

x = np.arange(len(labels))
bar_width = 0.35

plt.figure(figsize=(10, 4), dpi=100)
# pos bars (left offset)
plt.bar(
    x - bar_width/2,
    acc['pos'],
    bar_width,
    label='using zero vel. onsets',
    color='tab:blue',
    # edgecolor='gray'
)
# vel bars (right offset)
plt.bar(
    x + bar_width/2,
    acc['vel'],
    bar_width,
    label='using peak vel. onsets',
    color='tab:orange',
    # edgecolor='gray'
)

# Labels & titles
plt.xticks(x, labels, rotation=60)
plt.ylabel('Accuracy (%)')
plt.title(f'Plot 2: Tempo Estimation Accuracy: Zero Vel. vs Zero Vel. Onsets , {mode}')
plt.ylim(0, 100)
plt.legend()

# Annotate each bar
for idx in range(len(x)):
    plt.text(
        x[idx] - bar_width/2, acc['pos'][idx] + 1,
        f"{acc['pos'][idx]}%", ha="center", va="bottom", fontsize=9
    )
    plt.text(
        x[idx] + bar_width/2, acc['vel'][idx] + 1,
        f"{acc['vel'][idx]}%", ha="center", va="bottom", fontsize=9
    )

plt.tight_layout()
# plt.savefig("plots/plot_2a_multi-axis.png")
plt.show()


### V1: symmetric + symmetric body segment

In [ ]:
segments_3 = [
        # "both_hand_x", "both_hand_y", "both_foot_x", "both_foot_y",
        #  "both_hand_resultant", "both_foot_resultant", 
         
         "bothhand_x_bothfoot_x", "bothhand_y_bothfoot_y",
        "lefthand_xy_righthand_xy", "leftfoot_xy_rightfoot_xy",
        "bothhand_x_bothhand_y", "bothfoot_x_bothfoot_y", 
        ]  

a = 60; b= 140
metric = "pos"
mode = "zero_uni"
w_sec = 5; h_sec = w_sec/2

t_data = estimate_tempo_one(a, b, mode, metric, w_sec, h_sec)

accuracies = [round(t_data['bpm_median'][seg]['Acc1_bpm_one']) for seg in segments_3]

# Define labels
labels = [
    # 'Both-Hand-X', 'Both-Hand-Y', 'Both-Foot-X', 'Both-Foot-Y',
    #  'Both-Hand-Res', 'Both-Foot-Res',
     
     "bothhand_x_bothfoot_x", "bothhand_y_bothfoot_y",
        "lefthand_xy_righthand_xy", "leftfoot_xy_rightfoot_xy",
        "bothhand_x_bothhand_y", "bothfoot_x_bothfoot_y", 
    ]   

bar_colors = [
    '#AEC6CF', '#8FB1C2',  
    '#C3B1E1', '#A28DC8',  
    '#B0DAB9', '#92C9A0', 
    '#FFDAC1', '#E8BEA3',  
]

# Parameter to control bar width
bar_width = 0.15

# Number of pairs
n_pairs = len(labels) // 2

# Positioning pairs
pair_spacing = 0.15  # space between pairs
inner_spacing = 0.05  # gap between bars within a pair

# Calculate positions for bars to group them in pairs
positions = []
for i in range(n_pairs):
    start_pos = i * (2 * bar_width + pair_spacing)
    positions.extend([start_pos, start_pos + bar_width + inner_spacing])

x = np.array(positions)

# Plot
plt.figure(figsize=(10, 5))
bars = plt.bar(x, accuracies, color=bar_colors, edgecolor='gray', width=bar_width)

# Customize plot
plt.xticks(x, labels, rotation=60)
plt.ylabel('Accuracy (%)')
plt.title('Plot 3: Comparison of Tempo Estimation Accuracy Across Paired Hand/Foot Segment')
plt.ylim(0, 100)

# Annotate bar heights
for bar in bars:
    plt.text( bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{bar.get_height()}%', ha='center', fontsize=9 )

plt.tight_layout()
plt.show()

### V2: Symmetric body segment

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

segments_3 = [
    "both_hand_x", "both_hand_y",
    "both_foot_x", "both_foot_y",
    "both_hand_resultant", "both_foot_resultant",
]

a, b         = 60, 140
mode         = "zero_uni"
w_sec, h_sec = 5, 5/2

# compute pos accuracies
t_pos = estimate_tempo_one(a, b, mode, "pos", w_sec, h_sec, tolerance=0.13)
acc_pos = [round(t_pos['bpm_median'][seg]['Acc1_bpm_one']) for seg in segments_3]

# compute vel accuracies
t_vel = estimate_tempo_one(a, b, mode, "vel", w_sec, h_sec, tolerance=0.13)
acc_vel = [round(t_vel['bpm_median'][seg]['Acc1_bpm_one']) for seg in segments_3]

labels = [
    'Both-Hand-X', 'Both-Hand-Y',
    'Both-Foot-X', 'Both-Foot-Y',
    'Both-Hand-Res','Both-Foot-Res',
]

gap   = 0.2  
bar_width = 0.25# reduce this to bring bars closer
group = bar_width*2 + gap          # two bars per group + small gap
x = np.arange(len(labels)) * group
x = np.arange(len(labels)) * (bar_width*2 + 0.2)
# x = np.arange(len(labels))
# bar_width = 0.25

plt.figure(figsize=(8, 5), dpi=100)

# pos in blue
plt.bar(
    x - bar_width/2,
    acc_pos,
    bar_width,
    color='tab:blue',
    # edgecolor='gray',
    label='using zero vel. onsets',
)

# vel in orange
plt.bar(
    x + bar_width/2,
    acc_vel,
    bar_width,
    color='tab:orange',
    # edgecolor='gray',
    label='using peak vel. onsets',
)

plt.xticks(x, labels, rotation=60)
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)
plt.title('Plot 3: Accuracy for uni-directional (zero vel. vs peak vel onsets)')
plt.legend()

# annotate
for i in range(len(x)):
    plt.text(x[i] - bar_width/2, acc_pos[i] + 1,
             f"{acc_pos[i]}%", ha='center', va='bottom')
    plt.text(x[i] + bar_width/2, acc_vel[i] + 1,
             f"{acc_vel[i]}%", ha='center', va='bottom')

plt.tight_layout()
# plt.savefig("plots/plot_3a_both.png")
plt.show()


In [ ]:
for seg in segments_3:
    print(t_data['bpm_median'][seg]['Acc1_bpm_pos'])